# Annotations Gradio App

This notebook contains the code for running a gradio app for annotations. This can be migrated to Hugging Face Spaces if a persistant instance is required.

In [ ]:
import gradio as gr
import random
import os
import glob
import csv

In [ ]:
root_directory = 'results/'
models = ["SDXL", "pixart"]

CSV_FILE_PATH = f'{root_directory}annotations.csv'
MODEL_ORDER_PATH = f'..resources/annotations_model_order.txt'

In [ ]:
def log_folder_selection(user_name, folder_name, selection_choice):
    """Log the user's name, folder index, and selection choice to a CSV file."""
    file_exists = os.path.isfile(CSV_FILE_PATH)
    with open(CSV_FILE_PATH, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        if not file_exists:
            writer.writerow(['User Name', 'Prompt', 'Selection Choice'])  # Adding User Name as a header
        writer.writerow([user_name, folder_name, selection_choice])

# Get unique folder names across models in sorted order for consistent access
common_folder_names = []  # This will be a sorted list of common folder names
for model in models:
    model_folder_names = set()
    for folder_path in glob.glob(os.path.join(root_directory, model, "*")):
        folder_name = os.path.basename(folder_path)
        model_folder_names.add(folder_name)
    if not common_folder_names:
        common_folder_names = model_folder_names
    else:
        common_folder_names = common_folder_names.intersection(model_folder_names)

common_folder_names = sorted(list(common_folder_names))

#For Debug purposes
#common_folder_names = common_folder_names[0:3]

def fetch_images(folder_name):
    model1_image_paths = os.listdir(os.path.join(root_directory, models[0], folder_name))
    model2_image_paths = os.listdir(os.path.join(root_directory, models[1], folder_name))

    # Sort the paths to ensure consistency in which images are displayed
    model1_image_paths.sort()
    model2_image_paths.sort()

    # Only keep the first 20 images for each model
    model1_images = [os.path.join(root_directory, models[0], folder_name, img_name) for img_name in model1_image_paths[:20]]
    model2_images = [os.path.join(root_directory, models[1], folder_name, img_name) for img_name in model2_image_paths[:20]]

    return model1_images, model2_images


# Initialize galleries with first set of images if available
initial_folder_name = common_folder_names[0] if common_folder_names else ""
initial_model1_images, initial_model2_images = fetch_images(initial_folder_name) if initial_folder_name else ([], [])

# Load the model order from the file
with open(MODEL_ORDER_PATH, 'r') as f:
    models_order_list = [line.strip().split(',') for line in f.readlines()]


with gr.Blocks() as demo:
    folder_index_state = gr.State(1)
    #gr.Label("Consistency User Research")
    gr.Markdown(
    """
    # Consistency Annotation

    ### Instructions:
    The purpose of this annotation is to evaluate the consistency of different image generation models. 
    
    On one side of the screen, you will see the gallery from one model and on the other side of the screen will be the gallery from another model. 
    Your job is to evaluate how consistent, or similar, the images in each gallery are and determine which model gallery contains images that are the most similar to one another. 
    
    This should take 15-20 minutes to complete. You should not spend more than a few seconds on each entry.
    
    At the end, you will get a message letting you know you have finished (note-this will look like an Error). 
    
    If you receive a timeout error message, please just click the Next button.
    """)
    with gr.Row():
      gallery1 = gr.Gallery(value=initial_model1_images, columns=6, interactive=False, show_download_button=False)
      gallery2 = gr.Gallery(value=initial_model2_images, columns=6, interactive=False, show_download_button=False)
    user_name = gr.Textbox(label="Enter a unique id to be associated with your responses (can be your name):")
    feedback = gr.Radio(["left gallery", "right gallery"], label="Which set of images is more consistent?", info="(Consistency=more similar between images)")
    next_button = gr.Button("Next Set of Images")

    def update_galleries(_, folder_index, user_name, selection_choice):

      if folder_index < len(common_folder_names):
          print(folder_index)
          previous_folder_name = common_folder_names[folder_index - 1] if folder_index > 0 else common_folder_names[-1]
          previous_model_order = models_order_list[folder_index-1]
          selected_model_name = previous_model_order[0] if selection_choice == "left gallery" else previous_model_order[1]
          log_folder_selection(user_name, previous_folder_name, selected_model_name)

          models_order = models_order_list[folder_index]
          folder_name = common_folder_names[folder_index]
          model1_images, model2_images = fetch_images(folder_name)

          # Assign images to galleries based on the predefined model order
          if models_order[0] == "SDXL":
              images_for_galleries = [model1_images, model2_images]
          else:
              images_for_galleries = [model2_images, model1_images]

          # Increase folder_index for the next iteration, wrapping around if necessary
          folder_index += 1 #= (folder_index + 1) % len(common_folder_names)

          return images_for_galleries[0], images_for_galleries[1], folder_index

      else:
          raise gr.Error("Thanks for annotating! You may close this window.")



    next_button.click(
        update_galleries,
        inputs=[next_button, folder_index_state, user_name, feedback],
        outputs=[gallery1, gallery2, folder_index_state]
    )

demo.launch()